# Semana 6 — Fundamentos de Aprendizado de Máquina

Este notebook cobre um exercício guiado para visualizar underfitting e overfitting em um problema de regressão, avaliar regularização L2 e registrar boas práticas de preparo e avaliação de modelos.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error

plt.style.use('seaborn-v0_8')
np.random.seed(42)


## Geração de dados sintéticos

Usamos um polinômio cúbico com ruído para ter uma relação não linear moderada. Dividimos em treino/validação/teste para avaliar generalização.

In [ ]:
def gerar_dados(n=160, ruido=3.5):
    X = np.random.uniform(-3, 3, size=n)
    y = 0.5 * X**3 - X**2 + 2 * X + np.random.normal(scale=ruido, size=n)
    return X.reshape(-1, 1), y

X, y = gerar_dados()
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
print(f"Tamanhos: treino={len(X_train)}, validacao={len(X_val)}, teste={len(X_test)}")


In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.scatter(X_train, y_train, alpha=0.6, label='treino')
ax.scatter(X_val, y_val, alpha=0.6, label='validacao')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.legend()
ax.set_title('Dados com ruido')
plt.show()


## Modelos com diferentes complexidades

Avaliaremos regressão linear com graus polinomiais 1, 3 e 8.
- Grau 1: provável underfitting.
- Grau 3: próximo da função geradora.
- Grau 8: alta flexibilidade, sujeito a overfitting.

In [ ]:
def treinar_avaliar(degree):
    modelo = Pipeline([
        ('poly', PolynomialFeatures(degree=degree, include_bias=False)),
        ('reg', LinearRegression())
    ])
    modelo.fit(X_train, y_train)
    def rmse(X_set, y_set):
        preds = modelo.predict(X_set)
        return mean_squared_error(y_set, preds, squared=False)
    return {
        'grau': degree,
        'rmse_treino': rmse(X_train, y_train),
        'rmse_valid': rmse(X_val, y_val),
        'rmse_teste': rmse(X_test, y_test)
    }, modelo

avaliacoes = []
modelos = {}
for grau in [1, 3, 8]:
    res, model = treinar_avaliar(grau)
    avaliacoes.append(res)
    modelos[grau] = model

pd.DataFrame(avaliacoes)


In [ ]:
grid = np.linspace(-3.5, 3.5, 200).reshape(-1, 1)
fig, axes = plt.subplots(1, 3, figsize=(15, 4), sharey=True)
for ax, grau in zip(axes, [1, 3, 8]):
    ax.scatter(X_train, y_train, color='gray', alpha=0.4, label='treino')
    ax.scatter(X_val, y_val, color='tab:orange', alpha=0.5, label='validacao')
    preds = modelos[grau].predict(grid)
    ax.plot(grid, preds, color='tab:blue')
    ax.set_title(f'Grau {grau}')
    ax.set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].legend()
fig.suptitle('Curvas ajustadas vs. dados')
plt.show()


## Curvas de aprendizado

Comparamos um modelo simples (grau 2) e um modelo complexo (grau 8) à medida que o tamanho de treino cresce. Overfitting aparece quando o erro de treino é baixo, mas o de validação permanece alto.

In [ ]:
def plot_learning_curve(degree, ax):
    modelo = Pipeline([
        ('poly', PolynomialFeatures(degree=degree, include_bias=False)),
        ('reg', LinearRegression())
    ])
    train_sizes, train_scores, val_scores = learning_curve(
        modelo, X_train, y_train, cv=5, train_sizes=np.linspace(0.1, 1.0, 8), scoring='neg_root_mean_squared_error'
    )
    train_rmse = -train_scores.mean(axis=1)
    val_rmse = -val_scores.mean(axis=1)
    ax.plot(train_sizes, train_rmse, label='treino')
    ax.plot(train_sizes, val_rmse, label='validacao')
    ax.set_title(f'Curva de aprendizado (grau {degree})')
    ax.set_xlabel('amostras de treino')
    ax.set_ylabel('RMSE')
    ax.legend()

fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharey=True)
plot_learning_curve(2, axes[0])
plot_learning_curve(8, axes[1])
plt.show()


## Regularização L2 (Ridge)

Aplicamos Ridge com grau 8 para suavizar coeficientes e reduzir overfitting. Comparamos com o modelo não regularizado.

In [ ]:
def avaliar_ridge(alpha):
    modelo = Pipeline([
        ('poly', PolynomialFeatures(degree=8, include_bias=False)),
        ('reg', Ridge(alpha=alpha))
    ])
    modelo.fit(X_train, y_train)
    def rmse(X_set, y_set):
        return mean_squared_error(y_set, modelo.predict(X_set), squared=False)
    return {
        'alpha': alpha,
        'rmse_treino': rmse(X_train, y_train),
        'rmse_valid': rmse(X_val, y_val),
        'rmse_teste': rmse(X_test, y_test)
    }, modelo

ridge_result, ridge_model = avaliar_ridge(alpha=5.0)
lin_result, lin_model = treinar_avaliar(8)
pd.DataFrame([lin_result | {'modelo': 'Linear'}, ridge_result | {'modelo': 'Ridge'}])[['modelo','alpha','rmse_treino','rmse_valid','rmse_teste']]


In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.scatter(X_train, y_train, color='gray', alpha=0.4, label='treino')
ax.scatter(X_val, y_val, color='tab:orange', alpha=0.5, label='validacao')
pred_lin = lin_model.predict(grid)
pred_ridge = ridge_model.predict(grid)
ax.plot(grid, pred_lin, color='tab:red', label='Grau 8 sem reg.')
ax.plot(grid, pred_ridge, color='tab:green', label='Grau 8 Ridge')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.legend()
ax.set_title('Efeito da regularizacao L2')
plt.show()


## Boas práticas registradas

- Separar dados em treino/validação/teste para medir generalização.
- Usar validação cruzada quando o conjunto é pequeno para reduzir variância das métricas.
- Monitorar curvas de aprendizado para diagnosticar under/overfitting.
- Incluir regularização (ex.: L2) ou reduzir complexidade quando o erro de validação é muito maior que o de treino.
- Normalizar/escala de atributos para algoritmos sensíveis à magnitude e para regularização atuar de forma equilibrada.
- Fixar semente aleatória para reprodutibilidade e registrar métricas de teste apenas após finalizar tuning.